In [9]:
%gui qt
from mayavi import mlab
import numpy as np
import time
import os


In [10]:
# for testing
mlab.test_contour3d_anim()

In [11]:
DATA_SET = "data_multi_beauty"


Now, the numpy array is num_frames x 3 x number of satellites

In [12]:
# # Tmp code to mix all three bodies
# data0 = data = np.load(f"data/data_0.npy")
# data1 = data = np.load(f"data/data_1.npy")
# data2 = data = np.load(f"data/data_2.npy")

# print(data0.shape)
# print(data1.shape)
# print(data2.shape)

In [13]:

# DATA_SCALE_FACTOR = radius_earth
DATA_SCALE_FACTOR = 6.378e6

# Lets divide everything by R, to get intution
# new_shape = list(data0.shape)
# new_shape.append(3)

data = np.load(f"data/{DATA_SET}.npy")

# data[:, : , 0] = data0
# data[:, : , 1] = data1
# data[:, : , 2] = data2
data = data/DATA_SCALE_FACTOR


print(data.shape)
print(data[:10])

(72000, 3, 10)
[[[-1.90772024 -0.43841035 -1.59968766 -1.92195951 -1.77302791
   -1.22017964 -1.84711099  1.55155392  1.94670026 -1.48556459]
  [-0.6005027  -1.95135757 -1.20041634 -0.55323743  0.92540371
   -1.58466452  0.76692958 -1.26201444 -0.4586481   1.33906603]
  [ 0.05256051  0.05256051  0.05256051  0.05256051  0.05256051
    0.05256051  0.05256051  0.05256051  0.05256051  0.05256051]]

 [[-1.90640267 -0.43810756 -1.59858283 -1.9206321  -1.77180337
   -1.21933693 -1.84583528  1.55048233  1.94535577 -1.48453858]
  [-0.60008796 -1.95000986 -1.19958727 -0.55285533  0.92476458
   -1.58357007  0.7663999  -1.26114282 -0.45833134  1.33814121]
  [ 0.10512102  0.10512102  0.10512102  0.10512102  0.10512102
    0.10512102  0.10512102  0.10512102  0.10512102  0.10512102]]

 [[-1.90376889 -0.4375023  -1.59637432 -1.91797867 -1.76935554
   -1.21765236 -1.84328518  1.54834028  1.94266818 -1.48248763]
  [-0.59925891 -1.94731584 -1.19792999 -0.55209154  0.92348698
   -1.5813823   0.76534109 -1

In [14]:
# Frame of reference setup
frame_x, frame_y, frame_z = np.mgrid[-10: 10: 2j,-10: 10: 2j,-10: 10: 2j]

mlab.clf()

# Uncomment the below line to make endpoints invisible
# mlab.points3d(frame_x, frame_y, frame_z, opacity=0)
mlab.points3d(frame_x, frame_y, frame_z, opacity=0.0)


In [15]:

# So as to get spatial visualization in mayavi
VISUAL_SCALE_FACTOR = 3

In [16]:
earth_obj = mlab.points3d(0,0,0, color = (1,0,0))


sat_init_pos = data[0] * VISUAL_SCALE_FACTOR
# size is 3 * num_satellites
print((sat_init_pos[0]))
print(sat_init_pos[1])
print(sat_init_pos[2])

sat_objs = mlab.points3d(
        sat_init_pos[0],
        sat_init_pos[1],
        sat_init_pos[2],
        # color = (0.5, 0.5, 0.5)
        scale_factor = 1
    )
    

# print(sat_init_pos)


sat_trajs =[
    mlab.plot3d(
        sat_init_pos[0, i],
        sat_init_pos[1, i],
        sat_init_pos[2, i],
    )

    for i in range(data.shape[2])

]
print(sat_trajs)


[-5.72316072 -1.31523106 -4.79906298 -5.76587852 -5.31908373 -3.66053893
 -5.54133297  4.65466175  5.84010079 -4.45669377]
[-1.8015081  -5.85407271 -3.60124903 -1.65971228  2.77621113 -4.75399355
  2.30078875 -3.78604331 -1.37594431  4.0171981 ]
[0.15768153 0.15768153 0.15768153 0.15768153 0.15768153 0.15768153
 0.15768153 0.15768153 0.15768153 0.15768153]
[<mayavi.modules.surface.Surface object at 0x000001117160DC70>, <mayavi.modules.surface.Surface object at 0x0000011171F50630>, <mayavi.modules.surface.Surface object at 0x000001110BB42B30>, <mayavi.modules.surface.Surface object at 0x000001110BB7E680>, <mayavi.modules.surface.Surface object at 0x000001110BBB81D0>, <mayavi.modules.surface.Surface object at 0x000001110BBEDD60>, <mayavi.modules.surface.Surface object at 0x000001110BBCC9A0>, <mayavi.modules.surface.Surface object at 0x000001110BC5ACC0>, <mayavi.modules.surface.Surface object at 0x000001110BC94810>, <mayavi.modules.surface.Surface object at 0x000001110BCD0400>]


In [17]:
# Lets once try mlab animate decorator...
# If it doesn't work, do with save_fig and delay and all
# @mlab.animate

# Not using decorator here, as I need inputs to this function...

def anim(moving_obj, data, num_frames = 50, obj_trajs = None, pics_folder_name = None):

    # decorator not working... Animation stops in between
    # Now again,it doesn't stop

    delay = 100

    # @mlab.animate(delay = 100)

    if not os.path.exists(pics_folder_name):
                os.makedirs(pics_folder_name)
    if pics_folder_name is None:
         pics_folder_name = "pics"

    for i in range(num_frames):
        moving_obj.mlab_source.x = data[i][0] * VISUAL_SCALE_FACTOR  
        moving_obj.mlab_source.y = data[i][1] * VISUAL_SCALE_FACTOR 
        moving_obj.mlab_source.z = data[i][2] * VISUAL_SCALE_FACTOR 

        # Trajectory is not working, bodies just dont animate ...

        if obj_trajs is not None:
            for traj_idx, obj_traj in enumerate(obj_trajs):
                obj_traj.mlab_source.reset(
                    x = data[:i+1, 0, traj_idx] * VISUAL_SCALE_FACTOR,
                    y = data[:i+1, 1, traj_idx] * VISUAL_SCALE_FACTOR,
                    z = data[:i+1, 2, traj_idx] * VISUAL_SCALE_FACTOR
                )

        # Uncomment below line, if using decorator
        # yield
        # uncomment below lines, if not using decorator
        mlab.process_ui_events()
        # This savefig is a cruel trick to re-create the image or smthing like that
        mlab.savefig('./{}/anim_{:03}.png'.format(pics_folder_name, i))

        # time.sleep(delay/1000)



In [18]:
num_frames_interest = 200


anim(sat_objs, data, 
     num_frames = num_frames_interest, 
     obj_trajs = sat_trajs,
     pics_folder_name =  f"pics/{DATA_SET}/"
       )


# Obtained good animation results! Creates a folder with pic names for the same